In [2]:
import numpy as np
import pandas as pd
import pickle
from bs4 import BeautifulSoup
import re
import requests
import unicodedata
import random
import time
from collections import defaultdict

In [3]:
df = pd.read_pickle("urls.pkl")
df

,cik,company_name,filed_at,form_type,linkToTxt,ticker,name,sector
0,1090872,"AGILENT TECHNOLOGIES, INC.",2020-12-17,10-K,https://www.sec.gov/Archives/edgar/data/109087...,A,Agilent Technologies Inc,Health Care
1,1090872,"AGILENT TECHNOLOGIES, INC.",2020-09-01,10-Q,https://www.sec.gov/Archives/edgar/data/109087...,A,Agilent Technologies Inc,Health Care
2,1090872,"AGILENT TECHNOLOGIES, INC.",2020-05-29,10-Q,https://www.sec.gov/Archives/edgar/data/109087...,A,Agilent Technologies Inc,Health Care
3,1090872,"AGILENT TECHNOLOGIES, INC.",2020-03-03,10-Q,https://www.sec.gov/Archives/edgar/data/109087...,A,Agilent Technologies Inc,Health Care
4,6201,American Airlines Group Inc.,2021-02-17,10-K,https://www.sec.gov/Archives/edgar/data/6201/0...,AAL,American Airlines Group,Industrials
...,...,...,...,...,...,...,...,...
2334,1555280,Zoetis Inc.,2021-02-16,10-K,https://www.sec.gov/Archives/edgar/data/155528...,ZTS,Zoetis,Health Care
2335,1555280,Zoetis Inc.,2020-11-05,10-Q,https://www.sec.gov/Archives/edgar/data/155528...,ZTS,Zoetis,Health Care
2336,1555280,Zoetis Inc.,2020-08-06,10-Q,https://www.sec.gov/Archives/edgar/data/155528...,ZTS,Zoetis,Health Care
2337,1555280,Zoetis Inc.,2020-05-06,10-Q,https://www.sec.gov/Archives/edgar/data/155528...,ZTS,Zoetis,Health Care


In [4]:
#create dict for webscrape below
form_dict = defaultdict(list)
for inx, row in df.iterrows():
    form_dict[row['ticker']].append(row['linkToTxt'])
form_dict

defaultdict(list,
            {'A': ['https://www.sec.gov/Archives/edgar/data/1090872/000109087220000020/0001090872-20-000020.txt',
              'https://www.sec.gov/Archives/edgar/data/1090872/000109087220000014/0001090872-20-000014.txt',
              'https://www.sec.gov/Archives/edgar/data/1090872/000109087220000010/0001090872-20-000010.txt',
              'https://www.sec.gov/Archives/edgar/data/1090872/000109087220000004/0001090872-20-000004.txt'],
             'AAL': ['https://www.sec.gov/Archives/edgar/data/6201/000000620121000014/0000006201-21-000014.txt',
              'https://www.sec.gov/Archives/edgar/data/6201/000000620120000101/0000006201-20-000101.txt',
              'https://www.sec.gov/Archives/edgar/data/6201/000000620120000089/0000006201-20-000089.txt',
              'https://www.sec.gov/Archives/edgar/data/6201/000000620120000066/0000006201-20-000066.txt',
              'https://www.sec.gov/Archives/edgar/data/6201/000000620120000023/0000006201-20-000023.txt'],
  

In [5]:
text = {}

for ticker, urls in form_dict.items():
    for url in urls:
        res = requests.get(url)
        page = res.content
        try: #try/except statement used due to parsing errors encountered with a handful of links
            soup = BeautifulSoup(page, 'html.parser')
        except:
            text[ticker] = "error"
            continue

        #depending on the financial statement, divs and/or paragraphs are used within the HTML to represent paragraphs
        paragraphs = [paragraph for paragraph in soup.find_all('p')]
        divs = [div for div in soup.find_all('div')]

        blobs = []

        for paragraph in paragraphs:
            if paragraph.find_all(text=re.compile('COVID')): #searching for all instances with text "COVID-19"
                for element in paragraph.find_all(text=re.compile('COVID')):
                    blobs.append(str(unicodedata.normalize('NFKD', element)))

        for div in divs:
            if div.find_all(text=re.compile('COVID')): #searching for all instances with text "COVID-19
                for element in div.find_all(text=re.compile('COVID')):
                    blobs.append(str(unicodedata.normalize('NFKD', element)))

        for blob in blobs: #using as hack to remove unwanted, unparsed HTML that was appended to 'blobs'
            if "font-family" in blob:
                blobs.remove(blob)

        text[ticker + ";" + url] = blobs #append to final dictionary

        time.sleep(.5+2*random.random()) #random sleep time set before moving to next link

/Users/dunleavyjason/opt/anaconda3/lib/python3.8/site-packages/bs4/builder/_htmlparser.py:102: UserWarning: expected name token at '<![.ES@?Q0%440L&FW^3'
  warnings.warn(msg)
/Users/dunleavyjason/opt/anaconda3/lib/python3.8/site-packages/bs4/builder/_htmlparser.py:102: UserWarning: expected name token at '<![.WUG]IKQC\\;=2FMHX'
  warnings.warn(msg)
/Users/dunleavyjason/opt/anaconda3/lib/python3.8/site-packages/bs4/builder/_htmlparser.py:102: UserWarning: expected name token at '<![2EMPNU>*R\nMAIOA56'
  warnings.warn(msg)
/Users/dunleavyjason/opt/anaconda3/lib/python3.8/site-packages/bs4/builder/_htmlparser.py:102: UserWarning: unknown status keyword 'KU' in marked section
  warnings.warn(msg)
/Users/dunleavyjason/opt/anaconda3/lib/python3.8/site-packages/bs4/builder/_htmlparser.py:102: UserWarning: expected name token at '<![=+Z*.Y5 JD2H#NQQ7'
  warnings.warn(msg)
/Users/dunleavyjason/opt/anaconda3/lib/python3.8/site-packages/bs4/builder/_htmlparser.py:102: UserWarning: unknown status 

/Users/dunleavyjason/opt/anaconda3/lib/python3.8/site-packages/bs4/builder/_htmlparser.py:102: UserWarning: unknown status keyword 'AWYJKH' in marked section
  warnings.warn(msg)
/Users/dunleavyjason/opt/anaconda3/lib/python3.8/site-packages/bs4/builder/_htmlparser.py:102: UserWarning: expected name token at '<![\nM6</4&_B=WG)Y81C'
  warnings.warn(msg)
/Users/dunleavyjason/opt/anaconda3/lib/python3.8/site-packages/bs4/builder/_htmlparser.py:102: UserWarning: expected name token at '<![?/6?,+]SB(MN7[BB$'
  warnings.warn(msg)
/Users/dunleavyjason/opt/anaconda3/lib/python3.8/site-packages/bs4/builder/_htmlparser.py:102: UserWarning: unknown status keyword 'C' in marked section
  warnings.warn(msg)
/Users/dunleavyjason/opt/anaconda3/lib/python3.8/site-packages/bs4/builder/_htmlparser.py:102: UserWarning: expected name token at '<![%$IBB8"B8P@73FJ#U'
  warnings.warn(msg)
/Users/dunleavyjason/opt/anaconda3/lib/python3.8/site-packages/bs4/builder/_htmlparser.py:102: UserWarning: unknown statu

In [6]:
with open('text_v4.pickle', 'wb') as handle:
    pickle.dump(text, handle, protocol=pickle.HIGHEST_PROTOCOL)